In [8]:
from custom_data import OxfordPetDataset
import custom_loader
import os
from models import ResNetBackbone, CNNBackbone, BboxHead, ClassifierHead
import torch 
import sys
import io

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device}")

loader_dict = {
    "breed":{},
    "species":{},
    "bbox":{}
}

#### Preparing loaders
print("Preparing data loaders")
dataset = OxfordPetDataset().prepare_dataset()

for mode in loader_dict.keys():
    train_loader, val_loader, test_loader = custom_loader.create_dataloaders(dataset, target_type=mode)
    loader_dict[mode]['train_loader'] = train_loader
    loader_dict[mode]['val_loader'] = val_loader
    loader_dict[mode]['test_loader'] = test_loader

print("")
print("Successfully prepared data loaders!")

Using mps
Preparing data loaders
Images already downloaded: oxford_pet_data/images.tar.gz
Annotations already downloaded: oxford_pet_data/annotations.tar.gz
Dataset prepared with 35 classes.
Dataset split complete: training (70.0%), validation (15.0%), testing (15.0%)
Dataset split complete: training (70.0%), validation (15.0%), testing (15.0%)
Dataset split complete: training (70.0%), validation (15.0%), testing (15.0%)

Successfully prepared data loaders!


In [12]:

backbone = CNNBackbone()
res_backbone = ResNetBackbone()
cnn_head = BboxHead(adapter="cnn")
res_head = BboxHead(adapter="res")
res_class_head = ClassifierHead(adapter="res")
cnn_class_head = ClassifierHead(adapter="cnn")

loader = loader_dict["bbox"]['train_loader']

### First, test whether backbones and heads fit together
for images, labels in loader:
    features = backbone(images)
    features2 = res_backbone(images)
    
    predictions = cnn_head(features)
    predictions2 = res_head(features2)

    predictions3 = res_class_head(features2)
    predictions4 = cnn_class_head(features)
    print("All tests passed.")
    break

All tests passed.
